## Домашнее задание к занятию "Классификация: Логистическая регрессия и SVM"
Преподаватель: Алексей Миронов, Наталья Баданина, Дарья Максимова
Имеются данные adult.csv (см. в материалах к занятию).
Целевой переменной является уровень дохода income (крайний правый столбец).
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.
Готовый ноутбук выложить на гитхаб и прислать ссылку.

In [42]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [43]:
data = pd.read_csv( 'adult.csv', na_values='?')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


Описание переменнных:
- age: continuous.
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt: continuous.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- class: >50K, <=50K

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [45]:
data.isna().sum()

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

In [46]:
# обнуляем ячейки с отсутствием данных
data = data[ pd.isna( data['workclass'] ) == 0 ]
data = data[ pd.isna( data['occupation'] ) == 0 ]
data = data[ pd.isna( data['native-country'] ) == 0 ]

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational-num  45222 non-null  int64 
 5   marital-status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  object
 10  capital-gain     45222 non-null  int64 
 11  capital-loss     45222 non-null  int64 
 12  hours-per-week   45222 non-null  int64 
 13  native-country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [48]:
data.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,45222.000000,4.522200e+04,45222.000000,45222.000000,45222.000000,45222.000000
mean,38.547941,1.897347e+05,10.118460,1101.430344,88.595418,40.938017
std,13.217870,1.056392e+05,2.552881,7506.430084,404.956092,12.007508
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.173882e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783160e+05,10.000000,0.000000,0.000000,40.000000
75%,47.000000,2.379260e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [49]:
#Для дальнейшего анализа выберем следующие признаки: возраст, образование в годах, отношения, расу, количество рабочих часов в неделю и доход.
selectedColumns = data[ [ 'age', 'educational-num', 'relationship', 'race','hours-per-week'] ]
target= data['income']
# столбец  relationship и race являются категориальными переменными
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'relationship', 'race'] )
X.head()

,age,educational-num,hours-per-week,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,7,40,0,0,0,1,0,0,0,0,1,0,0
1,38,9,50,1,0,0,0,0,0,0,0,0,0,1
2,28,12,40,1,0,0,0,0,0,0,0,0,0,1
3,44,10,40,1,0,0,0,0,0,0,0,1,0,0
5,34,6,30,0,1,0,0,0,0,0,0,0,0,1


In [50]:
target.head() 

0    <=50K
1    <=50K
2     >50K
3     >50K
5    <=50K
Name: income, dtype: object

In [11]:
X.shape


(45222, 14)

In [12]:
target.shape

(45222,)

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   age                          45222 non-null  int64
 1   educational-num              45222 non-null  int64
 2   hours-per-week               45222 non-null  int64
 3   relationship_Husband         45222 non-null  uint8
 4   relationship_Not-in-family   45222 non-null  uint8
 5   relationship_Other-relative  45222 non-null  uint8
 6   relationship_Own-child       45222 non-null  uint8
 7   relationship_Unmarried       45222 non-null  uint8
 8   relationship_Wife            45222 non-null  uint8
 9   race_Amer-Indian-Eskimo      45222 non-null  uint8
 10  race_Asian-Pac-Islander      45222 non-null  uint8
 11  race_Black                   45222 non-null  uint8
 12  race_Other                   45222 non-null  uint8
 13  race_White                   45222 non-null  u

In [14]:
# целевая переменная (столбец income)  является категориальной
# переведем значения столбца в числа 0 и 1 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() 

In [15]:
le.fit(target)

LabelEncoder()

In [16]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [17]:
le.transform( [ '<=50K', '>50K', '>50K' ] )

array([0, 1, 1], dtype=int64)

In [18]:
# записываем в переменную y преобразованный столбец income
y = pd.Series(data = le.transform(target))
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

## модель логистической регрессии

In [19]:
model = LogisticRegression()

In [20]:
X[['relationship_Husband', 'relationship_Not-in-family','relationship_Other-relative','relationship_Own-child','relationship_Unmarried','relationship_Wife','race_Amer-Indian-Eskimo','race_Asian-Pac-Islander','race_Black','race_Other', 'race_White']] = X[['relationship_Husband', 'relationship_Not-in-family','relationship_Other-relative','relationship_Own-child','relationship_Unmarried','relationship_Wife','race_Amer-Indian-Eskimo','race_Asian-Pac-Islander','race_Black','race_Other', 'race_White']].astype(int)

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   age                          45222 non-null  int64
 1   educational-num              45222 non-null  int64
 2   hours-per-week               45222 non-null  int64
 3   relationship_Husband         45222 non-null  int32
 4   relationship_Not-in-family   45222 non-null  int32
 5   relationship_Other-relative  45222 non-null  int32
 6   relationship_Own-child       45222 non-null  int32
 7   relationship_Unmarried       45222 non-null  int32
 8   relationship_Wife            45222 non-null  int32
 9   race_Amer-Indian-Eskimo      45222 non-null  int32
 10  race_Asian-Pac-Islander      45222 non-null  int32
 11  race_Black                   45222 non-null  int32
 12  race_Other                   45222 non-null  int32
 13  race_White                   45222 non-null  i

In [22]:
X.describe()

,age,educational-num,hours-per-week,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
count,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000
mean,38.547941,10.118460,40.938017,0.412764,0.258768,0.029831,0.146522,0.105878,0.046239,0.009619,0.028813,0.093494,0.007806,0.860267
std,13.217870,2.552881,12.007508,0.492336,0.437963,0.170122,0.353632,0.307684,0.210004,0.097606,0.167284,0.291127,0.088007,0.346714
min,17.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,9.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,37.000000,10.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,47.000000,13.000000,45.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,90.000000,16.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train,y_test=train_test_split(X,y, test_size=0.2,random_state=42)

In [25]:
X_train.shape, y_train.shape

((36177, 14), (36177,))

In [26]:
y_train.head()

7963     1
26402    1
31411    1
13367    0
38742    0
dtype: int32

In [27]:
# обучаем модель на части датасета (train)
model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [28]:
predictions = model.predict_proba( X_test )

In [29]:
predictions[:5] # посмотрим как выглядят наши предсказания

array([[0.99393397, 0.00606603],
       [0.7131579 , 0.2868421 ],
       [0.50193894, 0.49806106],
       [0.96251619, 0.03748381],
       [0.63765774, 0.36234226]])

In [30]:

zip( predictions[:, 1], y )

In [31]:
model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

Получаем наш скор (точность предсказания) на обучающей и тестовой выборках.

In [32]:
model.score(X_test, y_test)

0.8187949143173023

In [34]:
model.score(X_train, y_train)

0.8161539099427814

При этом точность является хорошей и для обучающей и для тестовой выборки

## Модель SVM

In [51]:
from sklearn.svm import SVC

In [54]:
model2 = SVC()
model2.fit(X_train,y_train)

SVC()

In [57]:
model2.score(X_train, y_train)

0.813500290239655

In [58]:
model2.score(X_test, y_test)

0.8202321724709785

Тоже хорошая точность как для обучающей, так и для тестовой выборки. Даже чуть лучше, чем у логистической регрессии